In [1]:
def group_by_patterns(statement, patterns):
    i=0
    groups = []
    patterns = sorted(patterns, key=lambda k: -len(k))
    
    while i<len(statement):
        match = None
        for pattern in patterns:
            begin = i
            end = i+len(pattern)
            if pattern == statement[begin:end]:
                match = (begin, end)
                i=end-1
                break

        if match is not None:
            groups.append( match )
        else:
            groups.append( (i, i+1) )
        i+=1
    return groups

statement = ["red", "green", "blue", "A", "B", "G", "R", "Z", "u", "v", "w", "u", "v", "x", "y", "k", "B", "G", "R"]
patterns = ["red", "green", "blue"], ["A", "B", "G", "R"], ["B", "G", "R"], ["x", "y"], ["u", "v"], ["u", "v", "w"]

for span in group_by_patterns(statement, patterns):
    print(span, statement[span[0]:span[1]])

(0, 3) ['red', 'green', 'blue']
(3, 7) ['A', 'B', 'G', 'R']
(7, 8) ['Z']
(8, 11) ['u', 'v', 'w']
(11, 13) ['u', 'v']
(13, 15) ['x', 'y']
(15, 16) ['k']
(16, 19) ['B', 'G', 'R']


In [2]:
# reference for match span

import re
statement = "ABGRZuvxykBGR"
matches = [m for m in re.finditer(r'(ABGR)|(BGR)|(uvw)|(uv)|(xy)', statement)]
for match in matches:
    print(match)

matches[0].span()

<re.Match object; span=(0, 4), match='ABGR'>
<re.Match object; span=(5, 7), match='uv'>
<re.Match object; span=(7, 9), match='xy'>
<re.Match object; span=(10, 13), match='BGR'>


(0, 4)

In [3]:
class Layer:
    def __init__(self, name):
        self.name = name
        self.channels = []

    def __repr__(self):
        return "Layer('{}', {})".format(self.name, ", ".join(self.channels))

    @staticmethod
    def split_channel_id(channel_id):
        tags = channel_id.split(".")
        viewlayer_name = ".".join(tags[0:-1]) if len(tags)>1 else ""
        channel_name = tags[-1]
        return (viewlayer_name, channel_name)

    def group_by_delimiters(self):
        sub_layers = []
        
        for i in range(len(self.channels)):
            channel_id = cl[i]
            viewlayer, channel = Layer.split_channel_id(channel_id)

            if len(sub_layers)==0:
                sub_layers.append(Layer(name=viewlayer))

            if sub_layers[-1].name == viewlayer:
                pass
            else:
                sub_layers.append(Layer(name=viewlayer))

            # print(" append channel:", channel)
            sub_layers[-1].channels.append(channel)

        return sub_layers

    def group_by_patterns(self, patterns):
        sub_layers = []
        cl = self.channels
        for span in group_by_patterns(cl, patterns):
            begin, end = span
            sub_layer = Layer(name=layer.name)
            sub_layer.channels = cl[begin:end]
            sub_layers.append(sub_layer)
        return sub_layers

cl = ["A",
"B",
"G",
"R",
"Z",
"disparityL.x",
"disparityL.y",
"disparityR.x",
"disparityR.y",
"forward.left.u",
"forward.left.v",
"forward.right.u",
"forward.right.v",
"left.A",
"left.B",
"left.G",
"left.R",
"left.Z",
"whitebarmask.left.mask",
"whitebarmask.right.mask"]

lyr = Layer(name="")
lyr.channels = cl
layers = lyr.group_by_delimiters()
print(lyr)
print("->")
print("layers: ")
for layer in layers:
    print(layer)

Layer('', A, B, G, R, Z, disparityL.x, disparityL.y, disparityR.x, disparityR.y, forward.left.u, forward.left.v, forward.right.u, forward.right.v, left.A, left.B, left.G, left.R, left.Z, whitebarmask.left.mask, whitebarmask.right.mask)
->
layers: 
Layer('', A, B, G, R, Z)
Layer('disparityL', x, y)
Layer('disparityR', x, y)
Layer('forward.left', u, v)
Layer('forward.right', u, v)
Layer('left', A, B, G, R, Z)
Layer('whitebarmask.left', mask)
Layer('whitebarmask.right', mask)


In [4]:

lyr = Layer(name="color")
lyr.channels = ["red", "green", "blue", "A", "B", "G", "R", "Z", "u", "v", "w", "u", "v", "x", "y", "k", "B", "G", "R"]
def Layer_group_by_patterns(layer, patterns):
    sub_layers = []
    for span in group_by_patterns(layer.channels, patterns):
        begin, end = span
        sublayer = Layer(name=layer.name)
        sublayer.channels = layer.channels[begin:end]
        sub_layers.append(sublayer)
    return sub_layers

print(lyr.channels)

for sublyr in lyr.group_by_patterns(patterns):
    print(sublyr)

['red', 'green', 'blue', 'A', 'B', 'G', 'R', 'Z', 'u', 'v', 'w', 'u', 'v', 'x', 'y', 'k', 'B', 'G', 'R']
Layer('whitebarmask.right', red, green, blue)
Layer('whitebarmask.right', A, B, G, R)
Layer('whitebarmask.right', Z)
Layer('whitebarmask.right', u, v, w)
Layer('whitebarmask.right', u, v)
Layer('whitebarmask.right', x, y)
Layer('whitebarmask.right', k)
Layer('whitebarmask.right', B, G, R)
